In [1]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dot, Flatten
from tensorflow.keras.optimizers import Adam

In [2]:
with open('corpus_100k', 'r', encoding = 'utf-8') as file:
    text = file.read()

In [3]:
#Creating word_bag and choosing 300,000 of them, because the RAM couldn't handle
text.lower()
word_bag = text.split()[:10000]

In [4]:
uniques, freq = np.unique(word_bag, return_counts = True)

In [5]:
for i in range(len(word_bag)):
    empty = [0]*len(uniques)
    empty[list(uniques).index(word_bag[i])] = 1
    word_bag[i] = empty

In [6]:
# This function creates (input, target) word couples, with the window_size
def word_pairs(word_list, window_size):
    word_pairs = []
    for i in range(window_size, len(word_list)-window_size):
        for j in range(1,window_size+1):
            word_pairs.append((word_list[i],word_list[i-j]))
            word_pairs.append((word_list[i],word_list[i+j]))

    return word_pairs

In [7]:
word_pair = word_pairs(word_bag,2)

In [8]:
#distribution of words with frequencies
def distribution(word_bag, uniques, freq):
  word_freq = {word: frequency/len(word_bag) for word, frequency in zip(uniques, freq)}

  return np.array([word_freq[word] for word in uniques])

In [9]:
#Randomly choose negative targets.
def negative_sampling(n_negative, word_bag, uniques, freq):
  negative_targets = []
  dist = distribution(word_bag, uniques, freq)
  for n in range(n_negative):
    target = [0] * len(uniques)
    indx = np.random.choice(range(len(uniques)), p=dist)
    target[indx] = 1
    negative_targets.append(target)

  return negative_targets



In [10]:
def train_data_generator(word_pair, batch_size, uniques, freq):
  while True:
    batch_word_pairs = random.sample(word_pair, batch_size)
    input_words, target_words, labels = [], [], []

    for context_word, target_word in batch_word_pairs:
      input_words.append(context_word)
      target_words.append(target_word)
      labels.append(1)

      negative_samples = negative_sampling(20, word_bag, uniques, freq)
      for negative_word in negative_samples:
        input_words.append(context_word)
        target_words.append(negative_word)
        labels.append(0)

    X_target = np.array(target_words)
    X_context = np.array(input_words)
    y = np.array(labels)

    yield ([X_context, X_target], y)

In [16]:
inp = tf.keras.Input(shape = (1,))
target = tf.keras.Input(shape = (1,))
input_embedding = Embedding(input_dim=len(uniques), output_dim=300)(inp)
target_embedding = Embedding(input_dim=len(uniques), output_dim=300)(target)


dot_product = Dot(axes=-1)([input_embedding, target_embedding])


flat_output = Flatten()(dot_product)
model = tf.keras.Model(inputs=[inp, target], outputs=flat_output)



In [17]:
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01))

In [18]:
data = train_data_generator(word_pair, 1, uniques, freq)

In [19]:
model.fit(data, epochs=10, steps_per_epoch=len(word_pair))

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node 'binary_crossentropy/clip_by_value' defined at (most recent call last):
    File "C:\Users\danie\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\danie\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\danie\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\danie\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\danie\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\danie\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\danie\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self._